<a href="https://colab.research.google.com/github/takahiromiura/class_data_analysis_II_2025/blob/main/notebooks/%E5%89%8D%E5%87%A6%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前処理

何らかの方法で分析に用いるデータを入手したとして、それをそのまま使えるわけではありません。
データの加工や整形などを行う必要があり、これらを **前処理** (preprocessing) といったりします。
ここでは、そのうちのいくつかの方法を `pandas` を使って学びます。

この前処理を実装するのはとても時間がかかることが多いです。
データの細かい部分まで理解しておく必要があるからです。
また、前処理には非常に多くのトピックがあり、[前処理大全](https://gihyo.jp/book/2018/978-4-7741-9647-3) という本もあります。

In [1]:
import pandas as pd
from pandas import DataFrame, Series
# import numpy as np # 後でインポートするが、ここでしても問題ない

## 欠損値

### 欠損値とは

多くの場合、ある要素のデータが入っていないことがあります。
例えば、収入に対するアンケート項目には答えなくない人もいるでしょうし、機器の故障で特定の地域のデータが入らないこともあります。
これをまとめて **欠損値** (missing value) と呼びます。
必要に応じて、欠損値に対する処理を行うことがあります。

以下は、[e-stat](https://www.e-stat.go.jp/) からダウンロードした 2020 年度の産業別雇用数増減の [データ](https://www.e-stat.go.jp/dbview?sid=0003225718) の内訳を示しています。

同じデータをダウンロードするには、次の設定でダウンロードしてください。
デフォルトの設定から変更するものは太字にしています。

- **ダウンロード範囲**: ページ上部の選択項目 (年度次)
- ファイル形式: CSV形式(クロス集計表形式・Shift-JIS)
- **ヘッダの出力**: 出力しない
- **コードの出力**: 出力しない
- **補助コードの出力**: 出力しない
- 階層コードの出力**: 出力しない
- **凡例の出力**: 出力しない
- 注釈を表示する: チェックしない
- データがない行を表示しない: チェック
- データがない列を表示しない: チェック
- 桁区切り(,)の選択: 登録時の設定を使用する
- **特殊文字の選択**: Nullに置き換える

データを Colab に入れて読み込んでみます。
ファイル名は `employee_change.csv` とします。

In [2]:
data = pd.read_csv("/content/employee_change.csv")
data

,年度次,産業,/雇用者数の増減率,回答企業数【社】,△１５％以下【％】,△１５％超△１０％以下【％】,△１０％超△５％以下【％】,△５％超０％未満【％】,０％【％】,０％超５％未満【％】,５％以上１０％未満【％】,１０％以上１５％未満【％】,１５％以上【％】,階級値平均【％】
0,2020年度,全産業,NaN,"1,108",1.8,0.3,3.2,18.0,10.6,46.8,11.8,3.2,4.3,2.11
1,2020年度,製造業,NaN,498,0.8,0.2,2.8,18.5,12.4,49.6,10.2,2.2,3.2,1.95
2,2020年度,素材型製造業,NaN,148,1.4,NaN,2.0,16.2,11.5,53.4,10.1,2.0,3.4,2.09
3,2020年度,加工型製造業,NaN,203,0.5,0.5,2.5,20.2,10.3,50.7,10.3,3.0,2.0,1.88
4,2020年度,その他の製造業,NaN,147,0.7,NaN,4.1,18.4,16.3,44.2,10.2,1.4,4.8,1.89
5,2020年度,非製造業,NaN,610,2.6,0.3,3.4,17.5,9.2,44.6,13.1,3.9,5.2,2.25


`△１５％超△１０％以下【％】` というカラムに注目してください、数値以外に、`NaN` という文字が表示されているのが見えます。
`NaN` という表記は、文字列として `NaN` というものが入っているわけではありません。
Nan は Not A Number の略で、数値ではないことを表しています。

### 欠損値の確認

値が欠損値かどうかを調べるには、[`isna`](https://pandas.pydata.org/docs/reference/api/pandas.isna.html) メソッドまたは [`isnull`](https://pandas.pydata.org/docs/reference/api/pandas.isnull.html) メソッドを使います (どちらも同じ機能)。
これは、欠損値 (`None` も含みます) なら `True`、そうでないなら `False` を返します。
`DataFrame` と `Series` どちらにも実装されていますが、注目したいカラムの `Series` で使ってみます。

ライブラリによっては、`isna` と `isnull` が異なる挙動をする場合もあります。
注意してください。

In [3]:
COL_NAME = "△１５％超△１０％以下【％】"  # 注目したいカラム
data[COL_NAME]

,△１５％超△１０％以下【％】
0,0.3
1,0.2
2,NaN
3,0.5
4,NaN
5,0.3


In [4]:
data[COL_NAME].isnull()  # 欠損値かどうかの判定

,△１５％超△１０％以下【％】
0,False
1,False
2,True
3,False
4,True
5,False


In [5]:
data[COL_NAME].isna()

,△１５％超△１０％以下【％】
0,False
1,False
2,True
3,False
4,True
5,False


例えば、これに `sum` メソッドを適用すると、欠損値の数が分かります。
`True` や `False` を数値として扱う場合、`True` は 1, `False` は 0 として扱います。

In [6]:
data[COL_NAME].isnull().sum()

np.int64(2)

メソッドがオブジェクトを返す場合、さらにそのオブジェクトのメソッドを続けて適用することができます。
これをメソッドチェーンといいます。
上の例では、`isnull` メソッドが `Series` オブジェクトを返すので、続けて `Series` の `sum` メソッドを適用しています。
メソッドチェーンを行うことで、短いコードで様々な操作を組み合わせることが可能です。

逆に、欠損値ではないことを確かめるには、[`notna`](https://pandas.pydata.org/docs/reference/api/pandas.notna.html#pandas.notna) メソッドか [`notnull`](https://pandas.pydata.org/docs/reference/api/pandas.notnull.html) メソッドを用います。
欠損値ならば `False`、欠損値でないならば、`True` を返します。

In [7]:
data[COL_NAME].notnull()  # 欠損値ではないかの判定

,△１５％超△１０％以下【％】
0,True
1,True
2,False
3,True
4,False
5,True


### 欠損値の穴埋め

欠損値を扱うときは、*なぜ* 欠損値になっているかを考える必要があります。
基本的には、データ公開元に欠損値の規則が記載されていることが多いです。
例えば、e-stat では、`凡例表示` という部分をクリックすると、欠損値の規則が記載されています (参考データ: [URL](https://www.e-stat.go.jp/dbview?sid=0003225718))。
ここでの欠損値は、「皆無または定義上該当数字がないもの」ということでした。
要するに、この例では 0 の場合は欠損値になるようです。

平均値などを計算する場合、`pandas` ではデフォルトで欠損値を無視して計算します。
実際に見てみましょう。

In [8]:
print(data[COL_NAME])
data[COL_NAME].mean()

0    0.3
1    0.2
2    NaN
3    0.5
4    NaN
5    0.3
Name: △１５％超△１０％以下【％】, dtype: float64


np.float64(0.325)

欠損値でない値の平均は、$(0.3 + 0.2 + 0.5 + 0.3)/4 = 0.325$ なので、たしかに欠損値を無視していることがわかりました。

ちなみに、`mean` メソッドの引数として `skipna = False` を入力すると、データに欠損値がある場合には `nan` を返します。

ここでは、実際には欠損値は `0` なので、この平均値は過大に推定されています。
欠損値を別の値に穴埋めするには、`fillna` メソッドを用います。
`fillna` メソッドには、欠損値に埋める値 (`value`) を指定するか、`method` キーワードで、そのデータの直前 (上の行) の値 (欠損値ではないもの) を埋めるか (`ffill`)、直後の値を埋めるか (`bfill`) を指定します。

`0` で欠損値を埋めたいので、次のようにします。

In [9]:
data[COL_NAME].fillna(0)  # 欠損値を 0 で穴埋め

,△１５％超△１０％以下【％】
0,0.3
1,0.2
2,0.0
3,0.5
4,0.0
5,0.3


欠損値を 0 で埋めた後に、平均値を計算します。

In [10]:
data[COL_NAME].fillna(0).mean()

np.float64(0.21666666666666667)

### 欠損データの除外

欠損値が、例えばアンケートの未回答であることを表す場合、データから欠損値がある行 (サンプル) を除くことがあります。
`Series` にも同じメソッドがありますが、`DataFrame` だと [`dropna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) メソッドを用いることで、欠損値がある *行* を除外できます。

`subset` キーワードで指定することで、特定のカラムで欠損値がある場合のみデータを除外するようにできます (カラムなどで複数指定可能)。
また、指定した全てのカラムで欠損値がある場合のみ除外するか (`how` キーワードで指定)、一定以上のデータの欠損がある場合にみ除外するか (`thresh` キーワードで指定) など、細かい設定が可能です。
逆に、カラムの除外もできます (`axis` キーワードで指定)。

ここでは詳しく扱いませんが、データを除外するかどうかには細心の注意を払ってください。
データを除外してしまうことで、意図しない偏り (バイアス) のある結果になる可能性があります。

`dropna` メソッドを使うと、カラム `△１５％超△１０％以下【％】` のインデックス `2` と `4` のデータが除外された結果を返します。


In [11]:
print(data[COL_NAME].isna())
data.dropna(subset = COL_NAME) # △１５％超△１０％以下【％】で欠損値がある行を除外する

0    False
1    False
2     True
3    False
4     True
5    False
Name: △１５％超△１０％以下【％】, dtype: bool


,年度次,産業,/雇用者数の増減率,回答企業数【社】,△１５％以下【％】,△１５％超△１０％以下【％】,△１０％超△５％以下【％】,△５％超０％未満【％】,０％【％】,０％超５％未満【％】,５％以上１０％未満【％】,１０％以上１５％未満【％】,１５％以上【％】,階級値平均【％】
0,2020年度,全産業,NaN,"1,108",1.8,0.3,3.2,18.0,10.6,46.8,11.8,3.2,4.3,2.11
1,2020年度,製造業,NaN,498,0.8,0.2,2.8,18.5,12.4,49.6,10.2,2.2,3.2,1.95
3,2020年度,加工型製造業,NaN,203,0.5,0.5,2.5,20.2,10.3,50.7,10.3,3.0,2.0,1.88
5,2020年度,非製造業,NaN,610,2.6,0.3,3.4,17.5,9.2,44.6,13.1,3.9,5.2,2.25


## データ型変換

`pandas` では、データを読み込んだりして `Series` や `DataFrame` を作成すると、自動でそのデータ型を類推してくれます。
しかし、データの型を後から変換するのが必要な場合もあります。

よくあるのが、欠損値の処理の際です。
時には、欠損値が何らかのシンボルで表されることがあります。
例えば、[e-stat](https://www.e-stat.go.jp/) からデータをダウンロードするとき、デフォルトでは欠損値はハイフン `-` などの記号で表されます。

次は、2020 年度の産業別雇用数増減の[データ](https://www.e-stat.go.jp/dbview?sid=0003225718)を、デフォルトの欠損値のままでダウンロードした場合の結果です。

同じデータをダウンロードしたい場合、先ほどの設定のうち、**特殊文字の選択** は変更しないままにします。

ファイル名は `employee_change_symbol.csv` とします。

In [12]:
data_symbol = pd.read_csv("/content/employee_change_symbol.csv")
data_symbol

,年度次,産業,/雇用者数の増減率,回答企業数【社】,△１５％以下【％】,△１５％超△１０％以下【％】,△１０％超△５％以下【％】,△５％超０％未満【％】,０％【％】,０％超５％未満【％】,５％以上１０％未満【％】,１０％以上１５％未満【％】,１５％以上【％】,階級値平均【％】
0,2020年度,全産業,NaN,"1,108",1.8,0.3,3.2,18.0,10.6,46.8,11.8,3.2,4.3,2.11
1,2020年度,製造業,NaN,498,0.8,0.2,2.8,18.5,12.4,49.6,10.2,2.2,3.2,1.95
2,2020年度,素材型製造業,NaN,148,1.4,-,2.0,16.2,11.5,53.4,10.1,2.0,3.4,2.09
3,2020年度,加工型製造業,NaN,203,0.5,0.5,2.5,20.2,10.3,50.7,10.3,3.0,2.0,1.88
4,2020年度,その他の製造業,NaN,147,0.7,-,4.1,18.4,16.3,44.2,10.2,1.4,4.8,1.89
5,2020年度,非製造業,NaN,610,2.6,0.3,3.4,17.5,9.2,44.6,13.1,3.9,5.2,2.25


`△１５％超△１０％以下【％】` カラムの、前と例との違いが分かるでしょうか。
この例では、前の例で `NaN` と表記されていたものがハイフン `-` で表示されています。

この違いは、データ型に表れます。
前の例の場合と今回の例の場合での該当のカラムのデータ型を見てみます。

In [13]:
data[COL_NAME].dtype # 欠損値が空欄の場合

dtype('float64')

In [14]:
data_symbol[COL_NAME].dtype # 欠損値がハイフンの場合

dtype('O')


`dtype('O')` というのは、`object` 型であることを示しています。
つまり、欠損値がハイフン `-` で表されている場合、`object` 型になっており、何も値を入れないことを欠損値として表現した場合は、`float64` 型、つまり小数点のデータ型として扱われています。

なぜ欠損値の処理が重要なのかというと、平均などの集計に `object` 型は使えないからです。

In [15]:
# data_symbol[COL_NAME].mean() # 実行するとエラー

まずは、ハイフン `-` は文字列なので、欠損値のシンボルを欠損値 (`NaN`) に変えるか、数値に変える必要があります。
[`replace`](https://pandas.pydata.org/docs/reference/api/pandas.Series.replace.html) メソッドを用いて、指定した対応関係の値の変換を行えます。
対応関係の指定の仕方はいくつかありますが、辞書型でやるのが直感的だと思います。
辞書型でやる場合、データの値 `key` のものが、`value` の値に変換されます。

ここでは、ハイフン `-` を 0 に変換します。

欠損値に変換する場合は、[`numpy`](https://numpy.org/ja/) というライブラリの [`nan`](https://numpy.org/doc/stable/reference/constants.html) オブジェクトを指定します。
`numpy` は一般に `np` と略されてインポートされます。
`pandas` をインストールしたときに自動でインストールされているはずです。

`numpy` は数値計算に特化した Python のライブラリです。
最小二乗法や関数の解を求めるといったことも可能です。
また、`pandas` もデータの実装に `numpy` を使っています。

In [16]:
import numpy as np
data_symbol[COL_NAME].replace({"-": np.nan})

,△１５％超△１０％以下【％】
0,0.3
1,0.2
2,NaN
3,0.5
4,NaN
5,0.3



欠損値の記号を変換したままでは、データ型は `object` のままです。
データ型を変換するには、`Series` の `astype` メソッドでデータ型を指定して変換します。
データ型の指定には、`int` や `str` などの関数か、`dtype` の文字列リテラル (`int64` など) を使います。
`dtype` の種類は次を参照してください。

- `dtype` の種類: https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes

例えば、ここでは小数点型に変換したいので、`float` 関数を入れるか、`"float64"` というデータ型の文字列リテラルを入れます。


In [17]:
data_symbol[COL_NAME].replace({"-": 0}).astype(float)  # 関数で指定

,△１５％超△１０％以下【％】
0,0.3
1,0.2
2,0.0
3,0.5
4,0.0
5,0.3


In [18]:
data_symbol[COL_NAME].replace({"-": 0}).astype("float64")  # データ型の文字列で指定

,△１５％超△１０％以下【％】
0,0.3
1,0.2
2,0.0
3,0.5
4,0.0
5,0.3


数値型に変換するならば、`pandas` の `to_numeric` 関数 (メソッドではないことに注意!) の方が便利なこともあります。
`to_numeric` 関数は自動で適当な数値型に変換してくれます(小数点を含んでいれば小数点型に、整数だけなら整数型に)。

In [19]:
pd.to_numeric(data_symbol[COL_NAME].replace({"-": 0}))  # 適当な数値型に変換

,△１５％超△１０％以下【％】
0,0.3
1,0.2
2,0.0
3,0.5
4,0.0
5,0.3


欠損値への変換の場合、`pandas` の [`NA`](https://pandas.pydata.org/docs/user_guide/missing_data.html#na-semantics) オブジェクトを使って欠損値にすることもできますが、少し注意が必要です。
`NA` は、文字列などにも対応した欠損値表現なので、`object` 型に使うと、文字列の欠損値として扱われます。
したがって、`astype` で数値に変換しようとするとエラーになります。
`to_numeric` 関数は、この違いを上手く処理してくれ、数値型の欠損値に変換してくれます。

欠損値のシンボルは、時には記号ではなく数値で表されることもあります。
例えば、1 ~ 5 段階で答えるようなアンケート項目のデータのときに、それよりも大きい値 (基本的には `9` をつけることが多い) を欠損値のシンボルとして表すことがあります。
したがって、まず欠損値がどのように扱われているかをきちんと知ることがデータ分析では重要です。

アンケートなどを依頼するときなどでは、欠損値の規則もしっかりとルール化し、伝えておく必要があります。
そうしないと、担当者によって様々なシンボルが使われたりと、後で泣きを見ることになります。

## データの操作

ここでは、データの加工について少し触れていきます。

### 四則演算

`Series`, `DataFrame` 同士で四則演算をすることが可能です。
`Series` 同士の場合は、インデックスが同じ要素同士で演算が行われます。
長さが異なるもの同士でも演算が可能ですが、片方しかない要素の計算結果は、`NaN` が入ります。
両方のインデックスが組み合わさった `Series` が計算結果として返されます。

In [20]:
l = Series([1, 2, 3], index=["A", "B", "C"])
l

,0
A,1
B,2
C,3


In [21]:
l + l  # 同じインデックス同士で足し算

,0
A,2
B,4
C,6


In [22]:
l + Series([1, 3, 2], index=["A", "C", "B"])  # インデックスの順番は影響しない

,0
A,2
B,4
C,6


In [23]:
l + Series([1, 2, 3], index = ["A", "B", "D"]) # C と D は NaN になる

,0
A,2.0
B,4.0
C,NaN
D,NaN


In [24]:
l + Series([1, 2], index = ["A", "B"]) # 長さが異なっても計算可能

,0
A,2.0
B,4.0
C,NaN


`DataFrame` だと行 (インデックス) と列 (カラム) が同じもの同士で計算されます。

In [25]:
data = DataFrame(
    {
        "product": ["Apple", "Banana", "Carrot"],
        "sold": [1, 4, 3],
    }
)
data

,product,sold
0,Apple,1
1,Banana,4
2,Carrot,3


In [26]:
data + data

,product,sold
0,AppleApple,2
1,BananaBanana,8
2,CarrotCarrot,6


どれかの要素で計算ができない場合にはエラーになるので、基本的には `Series` で計算することが多いです。

また、`Series`, `DataFrame` に対する数値との演算は、各要素に対して適用されます。

In [27]:
Series([1, 2, 3], index=["A", "B", "C"]) * 3  # 各要素の値が 3 倍される

,0
A,3
B,6
C,9


### 条件式と条件分岐

ある値より高い場合には 1、そうでない場合は 0 などのダミー変数と言われるものを作る場合があります。
例えば、年収の値に応じて、平均以上なら 1 をとる高収入ダミーなどを作成したりします。

その前に、条件式について説明します。
四則演算と同様、数値、`Series` 同士、`DataFrame` 同士で等しいか `==`、等しくないか `!=` といった判定が可能です。
しかし、四則演算とは異なり `Series` 同士で比較するときには、インデックスが全て同じである必要があります。
なお、`DataFrame` 同士で比較するときには、インデックスとカラムどちらも同じである必要があります。

各要素に対する条件式の結果、`bool` 型が返ってきます。

In [28]:
l = Series([1, 2, 3])
r = Series([5, 2, 3])
print(l)
print(r)

0    1
1    2
2    3
dtype: int64
0    5
1    2
2    3
dtype: int64


In [29]:
l == 3  # 各要素に対して a == 3 かどうかを判別

,0
0,False
1,False
2,True


In [30]:
l == r  # 各要素に対して a == b かどうかを判別

,0
0,False
1,True
2,True


In [31]:
l != r  # 各要素に対して a != b かどうかを判別

,0
0,True
1,False
2,False



また、`bool` 型は `&` や `|` を使った演算が可能で、条件 A かつ条件 B を満たすか判別するには `&` を、条件 A または条件 B を満たすかどうかを判別するには `|` を使います。

例えば、次の商品カテゴリーのデータを例にとり、カテゴリーが果物かつ価格が150 円以下かどうか、カテゴリーが果物または価格が 150 以下かどうかを判別します。


In [32]:
data = pd.DataFrame(
    {
        "product": ["Apple", "Banana", "Carrot", "Milk", "Beef"],
        "price": [120, 200, 150, 200, 500],
        "category": ["fruit", "fruit", "vegetable", "dairy", "meat"],
    }
)

In [33]:
data["category"] == "fruit"  # 商品カテゴリーが果物

,category
0,True
1,True
2,False
3,False
4,False


In [34]:
(data["category"] == "fruit") & (
    data["price"] <= 150
)  # 商品カテゴリーが果物 かつ価格が 150 円以下

,0
0,True
1,False
2,False
3,False
4,False


In [35]:
(data["category"] == "fruit") | (
    data["price"] <= 150
)  # 商品カテゴリーが果物 または価格が 150 円以下

,0
0,True
1,True
2,True
3,False
4,False


`bool` 型同士で `&`, `|` を使うので、条件式を括弧で囲み、先に評価する必要があることに注意して下さい。

これを利用すると、ダミー変数を作成できます。
値が `bool` 型に 1 をかけると `True` は 1 に `False` は 0 にできます。
なので、例えば平均価格より高い商品なら 1 をとり、そうでないなら 0 をとるダミー変数は次のように作ります。

In [36]:
data["price"] >= data["price"].mean()  # 平均価格以上かどうか

,price
0,False
1,False
2,False
3,False
4,True


In [37]:
1 * (data["price"] >= data["price"].mean())  # 高価格ダミー変数

,price
0,0
1,0
2,0
3,0
4,1


`data["price"].mean()` は価格の平均値をとるので、`data["price"] >= data["price"].mean()` は各要素を数値と比較していることに等しいです。

この結果を元に新しいダミー変数をデータに追加するならば、次のようにします。

### `isin` メソッド

`Series` の [`isin`](https://pandas.pydata.org/docs/reference/api/pandas.Series.isin.html#pandas.Series.isin) メソッドを使うと、配列内の各要素のどれかを含むかどうかを判別できます。

In [38]:
data["category"].isin(["fruit", "meat"])  # 商品カテゴリーが果物か肉かを判別

,category
0,True
1,True
2,False
3,False
4,True


## より高度なフィルタリング

単一の条件によるフィルタリング、例えば商品カテゴリーが果物のデータを参照するには次のようにできました。

In [39]:
data[data["category"] == "fruit"]

,product,price,category
0,Apple,120,fruit
1,Banana,200,fruit


`data["category"] == "fruit"` は `bool` 型の `Series` です。

つまり、このフィルタリングの構文は、`True` であるインデックスの `DataFrame` だけを返していることがわかります。
なので、複数の条件を組み合わせたフィルタリングも可能です。
商品カテゴリーが果物 かつ価格が 150 円以下のデータを次のように取得可能です。


In [40]:
data[(data["category"] == "fruit") & (data["price"] <= 150)]

,product,price,category
0,Apple,120,fruit



また、`numpy` の [`where`](https://numpy.org/doc/stable/reference/generated/numpy.where.html) 関数を使うことで、条件に応じた値を入れることもできます。

`where` 関数は、以下のように使います。

```py
numpy.where(<condition>, <x>, <y>)
```

`<condition>` は条件式、`<x>` は条件式が `True` のインデックスに入れる値、`<y>` は条件式が `False` のインデックスに入れる値を示しています。

In [41]:
price = Series([100, 200, 300, 400, 500])
price

,0
0,100
1,200
2,300
3,400
4,500


In [42]:
np.where(
    price >= price.mean(), "high", "low"
)  # 価格に応じた値を返す

array(['low', 'low', 'high', 'high', 'high'], dtype='<U4')

## ピボットテーブル

以下のような、学生のテストスコアのデータがあるとします。

In [43]:
data = pd.DataFrame(
    {
        "学生ID": [1, 2, 3, 4, 5, 6, 7, 8],
        "クラス": ["A", "A", "B", "B", "A", "B", "A", "B"],
        "教科": ["数学", "国語", "数学", "国語", "数学", "国語", "数学", "国語"],
        "点数": [80, 75, 90, 85, 70, 80, 95, 90],
    }
)

クラス・教科ごとの平均点数を計算し、クラスごとに得意な教科が異なるかを分析したいとします。
[`pivot_table`](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html) メソッドはこのようなクロス集計を行うのに便利です。
`pivot_table` を使うことで、データを 2 つ以上のカテゴリーの組み合わせ毎の集計データに変換します。

`pivot_table` は以下の 4 つのキーワードを用います。
- `values`: 集計するデータのカラム名。
- `index`: ピボットテーブルの行になるキーを指定します（複数指定可能）。
- `columns`: ピボットテーブルの列になるキーを指定します（複数指定可能）。
- `aggfunc`: 適用したい集計関数を指定します。
  - デフォルトでは平均値 `mean` で、他にも合計 `sum`、観測数 `count` などがあります。

以下はコード例です。
B クラスは国語・数学どちらも A クラスよりも平均が高いことが分かります。

In [44]:
data.pivot_table(values="点数", index="クラス", columns="教科")

教科,国語,数学
クラス,,
A,75.0,81.666667
B,85.0,90.000000


## ソート・ランク

[`sort_values`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html) メソッドは、指定したカラムの値に応じてデータを並び替えます。

In [45]:
df = pd.DataFrame(
    {
        "name": ["Alice", "Bob", "Charlie", "David", "Eve"],
        "age": [13, 14, 12, 15, 13],
        "score": [85, 62, 75, 88, 90],
    }
)

In [46]:
df.sort_values("age")  # 年齢でソート

,name,age,score
2,Charlie,12,75
0,Alice,13,85
4,Eve,13,90
1,Bob,14,62
3,David,15,88


In [47]:
df.sort_values("score")  # 点数でソート

,name,age,score
1,Bob,14,62
2,Charlie,12,75
0,Alice,13,85
3,David,15,88
4,Eve,13,90


`ascending` キーワードでソートを昇順 (ascending order) か降順 (decending order) か指定できます。
デフォルトでは昇順で、値が低い方から高い方に並び替えられます。
`ascending = False` で降順にできます。

In [48]:
df.sort_values("age", ascending=False)  # 降順にソート

,name,age,score
3,David,15,88
1,Bob,14,62
0,Alice,13,85
4,Eve,13,90
2,Charlie,12,75


[`set_index`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html) は指定したカラムをインデックスにすることができます。

In [49]:
df.set_index("name")

,age,score
name,,
Alice,13,85
Bob,14,62
Charlie,12,75
David,15,88
Eve,13,90


[`sort_index`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_index.html) はインデックスに応じてデータをソートします。

In [50]:
df.set_index("name").sort_index()

,age,score
name,,
Alice,13,85
Bob,14,62
Charlie,12,75
David,15,88
Eve,13,90
